In [ ]:
#Importing necessary packages
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#Importing the data set
file_path = r'C:\Users\İbrahim\Downloads\Arrest.xlsx'
df = pd.read_excel(file_path)

#1,2. Examine the variables,their data types, the head and tail of the data frame. Check for any separation argument problem (“;” or “,”) of the data,
#the existence of header , NAs.
print(df.head())
print(df.tail())
print(df.info())
if not df.columns.empty:
    print("DataFrame has headers.")
else:
    print("DataFrame does not have headers.")


# 3.Check whether
# a. Column headers are values, not variable names.
if df.columns.astype(str).str.isnumeric().all():
    # Change the types of column headers to string
    df.columns = df.columns.astype(str)


# b. Check if multiple variables are stored in one column
multiple_variables = df.applymap(lambda x: isinstance(x, str) and ',' in x).any().any()
print(f"b. Multiple variables stored in one column: {multiple_variables}")

# c. Check if variables are stored in both rows and columns
variables_in_rows_and_columns = df.applymap(lambda x: isinstance(x, str)).any().any()
print(f"c. Variables stored in both rows and columns: {variables_in_rows_and_columns}")


# d. Multiple types of observational units are stored in the same table.
mixed_data_types = df.applymap(type).nunique().gt(1).any()
print(f"d. Multiple types of observational units stored in the same table: {mixed_data_types}")

# e. A single observational unit is stored in multiple tables
common_id_column = 'ARREST_ID'

duplicate_rows = df[df.duplicated(subset=[common_id_column], keep=False)]

if not duplicate_rows.empty:
    print("e. A single observational unit is stored in multiple rows.")
else:
    print("e. No single observational unit is stored in multiple rows.")


#4. Fixing the column names
df.columns = df.columns.str.title()

# 5. Dropping unnecessary columns. (The "Amount" column is unnecessary since all values are 0.)
df = df.drop(columns=['Amount'])

# 6. Remove the duplicates if it is not the nature of the data. 
df = df.drop_duplicates()

#9.	Be sure that all strings are in the same format (e.g. all in lower-case). If not, correct them.
df["Arrest Type"] = df["Arrest Type"].str.strip().str.title()
df["General Category Of The Arrest"] = df["General Category Of The Arrest"].str.strip().str.title()
df["Charge"] = df["Charge"].str.strip().str.title()
df["(Male/Female)"] = df["(Male/Female)"].str.strip().str.title()
df["Location"] = df["Location"].str.strip().str.title()
df['Location'] = df['Location'].str.replace('_', ' ')


# 14, 16. Search for possible outliers. If there are outliers, replace them with the mean. And search for the missing values.

Q1 = df["Age"].quantile(0.25)
Q3 = df["Age"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df.loc[(df["Age"] < lower_bound) | (df["Age"] > upper_bound), "Age"] = "NA"
df["Age"] = pd.to_numeric(df["Age"], errors='coerce')
mean_age = round(df["Age"].mean(skipna=True))
df["Age"].fillna(mean_age, inplace=True)

Q1 = df["How Many Hours He/She Work?"].quantile(0.25)
Q3 = df["How Many Hours He/She Work?"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df.loc[(df["How Many Hours He/She Work?"] < lower_bound) | (df["How Many Hours He/She Work?"] > upper_bound), "How Many Hours He/She Work?"] = "NA"
df["How Many Hours He/She Work?"] = pd.to_numeric(df["How Many Hours He/She Work?"], errors='coerce')
mean_H = round(df["How Many Hours He/She Work?"].mean(skipna=True))
df["How Many Hours He/She Work?"].fillna(mean_H, inplace=True)


df["General Category Of The Arrest"].fillna(df["General Category Of The Arrest"].mode()[0], inplace=True)
df["Charge"].fillna(df["Charge"].mode()[0], inplace=True)
df["Arrest Type"].fillna(df["Arrest Type"].mode()[0], inplace=True)
df["Location"].fillna(df["Location"].mode()[0], inplace=True)



# 12. Examine the data types again and be sure that numeric variables are float, categorical ones are object, and date is in date format. If not, correct it.
print(df.info())
df["Age"] = df["Age"].astype(float)
df["What Is The Income Of Individual?"] = df["What Is The Income Of Individual?"].astype(float)
df["How Many Hours He/She Work?"] = df["How Many Hours He/She Work?"].astype(float)
df["Arrest_Id"] = df["Arrest_Id"].astype(str)
df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')